In [67]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
from tensorflow import keras
tf.keras.backend.set_floatx('float32')
from tqdm import tqdm_notebook as tqdm

2.3.1


### Define a sampling function 

In [2]:
def sampling_function(n_interior):
    inner_sample = tf.random.uniform(shape = (1000,2), minval = 0, maxval = 1, dtype = 'float32') 
    return inner_sample

In [3]:
s = sampling_function(100)

In [4]:
s.numpy()

array([[0.89163566, 0.8441485 ],
       [0.11397707, 0.15773988],
       [0.6136919 , 0.49608195],
       ...,
       [0.44599855, 0.23882818],
       [0.05281591, 0.4475268 ],
       [0.3675604 , 0.08517408]], dtype=float32)

### Define loss functions

In [79]:
def loss_function(n_interior):
#     inner_sample = tf.Variable(initial_value = sampling_function(100))
    with tf.GradientTape() as t1:
        inner_sample = sampling_function(100)
        t1.watch(inner_sample)
        predictions = model(inner_sample)
        t1.watch(predictions)
        grad = t1.gradient(predictions, inner_sample)
    grad_norm = tf.add(tf.reduce_sum(tf.math.square(grad[:, 0])), tf.reduce_sum(tf.math.square(grad[:, 1])))
    loss = tf.add(0.5*grad_norm, tf.reduce_sum(predictions))
    return loss

### Define model architecture 

In [80]:
class SkipBlock(keras.layers.Layer):
    def __init__(self, units = 10):
        super(SkipBlock, self).__init__()
        self.units = units
    def build(self, input_shape):
        self.first_layer = keras.layers.Dense(10, activation = 'relu', use_bias = True)
        self.second_layer = keras.layers.Dense(10, activation = 'relu', use_bias = True)
    def call(self, x):
        y1 = self.first_layer(x)
        y2 = self.second_layer(y1)
        return tf.add(y2, x)

In [81]:
class DeepRitz(keras.Model):
    def __init__(self):
        super(DeepRitz, self).__init__()
        self.input_dense = keras.layers.Dense(10, use_bias = True)
        self.out_dense = keras.layers.Dense(1, use_bias = True)
        self.sb1 = SkipBlock(units = 10)
        self.sb2 = SkipBlock(units = 10)
        self.sb3 = SkipBlock(units = 10)
    def call(self, x):
        y1 = self.input_dense(x)
        y2 = self.sb1(y1)
        y3 = self.sb2(y2)
        y4 = self.sb3(y3)
        output = self.out_dense(y4)
        return output

### Train model 

In [82]:
model = DeepRitz()
opt = keras.optimizers.Adam(1e-3)
n_interior = 1000

In [83]:
@tf.function  # Make it fast.
def train_model(n_steps):
    for i in tqdm(range(n_steps)):
        with tf.GradientTape() as tape:
            loss_val = loss_function(100)
            tape.watch(loss_val)
            gradients = tape.gradient(loss_val, model.trainable_weights)
        opt.apply_gradients(zip(gradients, model.trainable_weights))
        tf.print("Iteration:", i, ", Loss:", loss_val)

### Test run 

In [84]:
train_model(10)


Iteration: 0 , Loss: 1263.32422
Iteration: 1 , Loss: 1217.92065
Iteration: 2 , Loss: 1162.50586
Iteration: 3 , Loss: 1081.92773
Iteration: 4 , Loss: 1045.79834
Iteration: 5 , Loss: 989.848572
Iteration: 6 , Loss: 933.498291
Iteration: 7 , Loss: 870.670959
Iteration: 8 , Loss: 828.505737
Iteration: 9 , Loss: 771.922424
